In [8]:
import numpy as np
import tensorflow as tf

In [3]:
def LoadData(filename, load2=True, load3=True):
  """Loads data for 2's and 3's
  Inputs:
    filename: Name of the file.
    load2: If True, load data for 2's.
    load3: If True, load data for 3's.
  """
  assert (load2 or load3), "Atleast one dataset must be loaded."
  data = np.load(filename)
  if load2 and load3:
    inputs_train = np.hstack((data['train2'], data['train3']))
    inputs_valid = np.hstack((data['valid2'], data['valid3']))
    inputs_test = np.hstack((data['test2'], data['test3']))
    target_train = np.hstack((np.zeros((1, data['train2'].shape[1])), np.ones((1, data['train3'].shape[1]))))
    target_valid = np.hstack((np.zeros((1, data['valid2'].shape[1])), np.ones((1, data['valid3'].shape[1]))))
    target_test = np.hstack((np.zeros((1, data['test2'].shape[1])), np.ones((1, data['test3'].shape[1]))))
  else:
    if load2:
      inputs_train = data['train2']
      target_train = np.zeros((1, data['train2'].shape[1]))
      inputs_valid = data['valid2']
      target_valid = np.zeros((1, data['valid2'].shape[1]))
      inputs_test = data['test2']
      target_test = np.zeros((1, data['test2'].shape[1]))
    else:
      inputs_train = data['train3']
      target_train = np.zeros((1, data['train3'].shape[1]))
      inputs_valid = data['valid3']
      target_valid = np.zeros((1, data['valid3'].shape[1]))
      inputs_test = data['test3']
      target_test = np.zeros((1, data['test3'].shape[1]))

  return inputs_train, inputs_valid, inputs_test, target_train, target_valid, target_test

In [157]:
inputs_train, inputs_valid, inputs_test, target_train, target_valid, target_test = LoadData('digits.npz')

In [302]:
target_train_ints = target_train.astype(np.int64)
newArr = []
for i in xrange(target_train_ints.shape[1]):
    curr_el = target_train_ints[0][i]
    newArr.append(np.array([curr_el, 1 - curr_el], np.int32))
    #print target_train_ints[0][i].shape
target_input = np.asarray(newArr)

In [311]:
x = tf.placeholder(tf.float32, [None, 256])
W1 = tf.Variable(tf.zeros([256, 30]))
b1 = tf.Variable(tf.zeros([30]))

W2 = tf.Variable(tf.zeros([30, 2]))
b2 = tf.Variable(tf.zeros([2]))

hidden_layer = tf.matmul(x, W1) + b1
output = tf.matmul(hidden_layer, W2) + b2

In [312]:
y = tf.nn.softmax(output)                     # Actual output of our network
y_ = tf.placeholder(tf.float32, [None, 2])    # Expected output of our network

In [313]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

In [314]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [315]:
init = tf.initialize_all_variables()

In [316]:
sess = tf.Session()
sess.run(init)

In [323]:
for i in xrange(50, 650, 50):
    sess.run(train_step, feed_dict={x: inputs_train.T[i-50:i], y_: target_input[i-50:i]})

In [324]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))